<a href="https://colab.research.google.com/github/ishaanharry/UCSD-DML-image-segmentation/blob/main/RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
# testing
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torchsummary import summary
import torchvision
import torch.optim as optim
from torch.autograd import Variable
from time import time
import torchvision.transforms as T
import torch.utils.data
from natsort import natsorted
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib.patches as mpatches
from matplotlib import patches
import time
from tqdm.notebook import tqdm


In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The code will run on GPU.


In [ ]:
from google.colab import drive
# This will prompt for authorization.

drive.mount('/content/drive/')
!ls '/content/drive/'

Mounted at /content/drive/
MyDrive


In [ ]:
# Import fasterrcnn model, and other libraries

# /content/drive/My Drive/Colab Notebooks/02456-deep-learning-with-PyTorch-4_Mini_Project
import sys
drive_path = '/content/drive/MyDrive/warwick_qu_dataset_released_2016_07_08/Warwick QU Dataset (Released 2016_07_08)/detection'
sys.path.insert(0, drive_path)
module_names = [ f.name for f in os.scandir(drive_path)]
# module_names
for i in module_names:
  if 'README' in i:
    continue
  i = i.replace('.py', '') 
  __import__(i)
  

FileNotFoundError: ignored

In [ ]:

class WarwickCellDataset(object):
    def __init__(self, root, transforms=None): # transforms
        self.root = root
        # self.transforms = transforms
        self.transforms=[]
        if transforms!=None:
          self.transforms.append(transforms)
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(natsorted(os.listdir(os.path.join(root, "image"))))
        print('imgs file names:', self.imgs)
        self.masks = list(natsorted(os.listdir(os.path.join(root, "mask"))))
        print('masks file names:', self.masks)
        # images = []
        # masks = []
        # image_names = [f.name for f in os.scandir(root)]
        # image_names = natsorted(image_names)
        # for image_name in image_names:
        #   if image_name.endswith('_anno.bmp'):
        #     masks.append(image_name)
        #   else:
        #     images.append(image_name)
        # print(images)
        # print(masks)
        # print('Length images:', len(images))
        # print('Length masks:', len(masks))
        # self.imgs = images
        # self.masks = masks
        # print('Length images:', len(self.imgs))
        # print('Length masks:', len(self.masks))

    def __getitem__(self, idx):
        # idx sometimes goes over the nr of training images, add logic to keep it lower
        if idx >= 80:
          idx = np.random.randint(80, size=1)[0]
        # print(idx)
        # load images ad masks
        # print('idx:', idx)
        img_path = os.path.join(self.root, "image", self.imgs[idx])
        # print('img_path', self.imgs[idx])
        mask_path = os.path.join(self.root, "mask", self.masks[idx])
        # print('mask_path', self.masks[idx])
        # print('img_path name:', self.imgs[idx])
        # img_path = os.path.join(self.root, self.imgs[idx])
        # print('mask_path name:', self.masks[idx])
        # mask_path = os.path.join(self.root, self.masks[idx])

        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        # print(num_objs)
        boxes = []
        for i in range(num_objs):
          pos = np.where(masks[i])
          xmin = np.min(pos[1])
          xmax = np.max(pos[1])
          ymin = np.min(pos[0])
          ymax = np.max(pos[0])
          # Check if area is larger than a threshold
          A = abs((xmax-xmin) * (ymax-ymin)) 
          # print(A)
          if A < 5:
            print('Nr before deletion:', num_objs)
            obj_ids=np.delete(obj_ids, [i])
            # print('Area smaller than 5! Box coordinates:', [xmin, ymin, xmax, ymax])
            print('Nr after deletion:', len(obj_ids))
            continue
            # xmax=xmax+5 
            # ymax=ymax+5

          boxes.append([xmin, ymin, xmax, ymax])

        # print('nr boxes is equal to nr ids:', len(boxes)==len(obj_ids))
        num_objs = len(obj_ids)
        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        for i in self.transforms:
          img = i(img)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["labels"] = labels # Not sure if this is needed
        
        # trans = transforms.ToTensor()
        # img = trans(img)
        # transform = []
        # transform.append(transforms.RandomHorizontalFlip(0.5))
        # transform.append(transforms.ToTensor())
        # transform = Compose(transform)
        # img, target = transform(img, target)

        # if self.transforms is not None:
        #   # for t in self.transforms:
        #   #   img = t(img)
        #   #   target = t(target)
        #     # print(img.size)
        #     img, target = self.transforms(img, target)

        return img.double(), target

    def __len__(self):
        return len(self.imgs)

In [ ]:
root_train = '/content/drive/MyDrive/warwick_qu_dataset_released_2016_07_08/Warwick QU Dataset (Released 2016_07_08)/train'
dataset_train = WarwickCellDataset(root_train, transforms=torchvision.transforms.ToTensor()) # get_transform(train=True)
dataset_train[0]

In [ ]:
# Define data loader for training set
data_loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=2, shuffle=True,collate_fn=lambda x:list(zip(*x)))
print(len(data_loader_train))

In [ ]:
# View some images and their bbox's in the training set.

images,labels=next(iter(data_loader_train))

def view(images,labels,n=2,std=1,mean=0):
    figure = plt.figure(figsize=(15,10))
    images=list(images)
    labels=list(labels)
    for i in range(n):
        out=torchvision.utils.make_grid(images[i])
        inp=out.cpu().numpy().transpose((1,2,0))
        inp=np.array(std)*inp+np.array(mean)
        inp=np.clip(inp,0,1)  
        ax = figure.add_subplot(2,2, i + 1)
        ax.imshow(images[i].cpu().numpy().transpose((1,2,0)))
        l=labels[i]['boxes'].cpu().numpy()
        l[:,2]=l[:,2]-l[:,0]
        l[:,3]=l[:,3]-l[:,1]
        for j in range(len(l)):
            ax.add_patch(patches.Rectangle((l[j][0],l[j][1]),l[j][2],l[j][3],linewidth=1.5,edgecolor='r',facecolor='none')) 

view(images=images,labels=labels,n=2,std=1,mean=0)

In [ ]:
#@title
# import torchvision
# from torchvision.models.detection import FasterRCNN
# from torchvision.models.detection.rpn import AnchorGenerator

# # load a pre-trained model for classification and return
# # only the features
# backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# # FasterRCNN needs to know the number of
# # output channels in a backbone. For mobilenet_v2, it's 1280
# # so we need to add it here
# backbone.out_channels = 1280

# # let's make the RPN generate 5 x 3 anchors per spatial
# # location, with 5 different sizes and 3 different aspect
# # ratios. We have a Tuple[Tuple[int]] because each feature
# # map could potentially have different sizes and
# # aspect ratios
# anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
#                                    aspect_ratios=((0.5, 1.0, 2.0),))

# # let's define what are the feature maps that we will
# # use to perform the region of interest cropping, as well as
# # the size of the crop after rescaling.
# # if your backbone returns a Tensor, featmap_names is expected to
# # be [0]. More generally, the backbone should return an
# # OrderedDict[Tensor], and in featmap_names you can choose which
# # feature maps to use.
# roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
#                                                 output_size=7,
#                                                 sampling_ratio=2)



# # put the pieces together inside a FasterRCNN model
# model = FasterRCNN(backbone,
#                    num_classes=2,
#                    rpn_anchor_generator=anchor_generator,
#                    box_roi_pool=roi_pooler)


In [ ]:
#@title
# import torchvision
# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

# num_classes = 2  # 1 class (cell) + background = 2 classes
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True) # use the torch model, pre-trained on coco dataset if set to True
# in_features = model.roi_heads.box_predictor.cls_score.in_features

# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
# # now get the number of input features for the mask classifier
# in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
# hidden_layer = 256
# # and replace the mask predictor with a new one
# model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
#                                                     hidden_layer,
#                                                     num_classes)
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model=model.to(device)

# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.001)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


num_classes = 2
# load an instance segmentation model pre-trained pre-trained on COCO
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# now get the number of input features for the mask classifier
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
# and replace the mask predictor with a new one
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                    hidden_layer,
                                                    num_classes)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model=model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)


In [ ]:
# sp = '/content/drive/MyDrive/warwick_qu_dataset_released_2016_07_08/Warwick QU Dataset (Released 2016_07_08)/trained_models'
# model_names = os.listdir(os.path.join(sp))
# model_names
def latest_model():
  sp = '/content/drive/MyDrive/warwick_qu_dataset_released_2016_07_08/Warwick QU Dataset (Released 2016_07_08)/trained_models'
  largest = 0
  model_names = os.listdir(os.path.join(sp))
  for i in model_names:
    nr = int(list(filter(str.isdigit, i))[0])
    if nr>largest:
      largest = nr
  return largest

latest_model()

In [ ]:
# Load previously trained model 
model_nr = latest_model() 
save_path = '/content/drive/MyDrive/warwick_qu_dataset_released_2016_07_08/Warwick QU Dataset (Released 2016_07_08)/trained_models/model'+str(model_nr)
model.load_state_dict(torch.load(save_path))
# model=model.to(device)

In [ ]:

# Perform training loop for n epochs
loss_list = []
n_epochs = 10
model.train()
for epoch in tqdm(range(n_epochs)):
    loss_epoch = []
    iteration=1
    for images,targets in tqdm(data_loader_train):

        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        model=model.double()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()       
        optimizer.step()
        # print('loss:', losses.item())
        # loss_epoch.append(losses.item())
        loss_epoch.append(losses.item())
        # Plot loss every 10th iteration

        plt.plot(list(range(iteration)), loss_epoch)
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        plt.show()
        iteration+=1
    loss_epoch_mean = np.mean(loss_epoch) 
    loss_list.append(loss_epoch_mean) 
    # loss_list.append(loss_epoch_mean)    
    print("Average loss for epoch = {:.4f} ".format(loss_epoch_mean))

model_nr = latest_model() + 1
save_path = '/content/drive/MyDrive/warwick_qu_dataset_released_2016_07_08/Warwick QU Dataset (Released 2016_07_08)/trained_models/model'+str(model_nr)
torch.save(model.state_dict(), save_path)

In [ ]:
loss_list2 = [1.7433, 1.0864, 0.8294, 0.6757, 0.5676]

In [ ]:
# Plot training loss
plt.plot(list(range(n_epochs)), loss_list, label='traning loss')
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
# Load previously trained model 
# model.load_state_dict(torch.load(save_path))

In [ ]:
root_test = '/content/drive/MyDrive/warwick_qu_dataset_released_2016_07_08/Warwick QU Dataset (Released 2016_07_08)/test'
dataset_test = WarwickCellDataset(root_test, transforms=torchvision.transforms.ToTensor()) # get_transform(train=True)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=2, shuffle=False,collate_fn=lambda x:list(zip(*x)))

In [ ]:
# images, targets=next(iter(data_loader_test))
# images = list(image.to(device) for image in images)
# targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
# images

In [ ]:
# Look at some of the images and predicted bbox's after training

images, targets=next(iter(data_loader_test))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

model=model.double()
model.eval()
output = model(images)

with torch.no_grad():
    view(images, output, 2)

In [ ]:
print(len(output[0]['boxes']))
print(len(output[0]['scores']))

In [ ]:
output[0]['boxes'][0]

In [ ]:
# torchvision.utils.make_grid(images[i])
for i in range(2):
  # out = output[i]['scores'].to('cpu')
  # out = out.detach().numpy()
  for j in range(len(output[i]['scores'])):
    if j < 0.7:
      output[i]['boxes'][j] = torch.Tensor([0,0,0,0])

In [ ]:
def view_mask(targets, output, n=2, cmap='Greys'):
    figure = plt.figure(figsize=(15,10))
    for i in range(n):
      # plot target (true) masks
      target_im = targets[i]['masks'][0].cpu().detach().numpy()
      for k in range(len(targets[i]['masks'])):
        target_im2 = targets[i]['masks'][k].cpu().detach().numpy()
        target_im2[target_im2>0.5] = 1
        target_im2[target_im2<0.5] = 0
        target_im = target_im+target_im2

      target_im[target_im>0.5] = 1
      target_im[target_im<0.5] = 0
      ax = figure.add_subplot(2,2, i+1)
      ax.imshow(target_im, cmap=cmap)
      # Plot output (predicted) masks
      output_im = output[i]['masks'][0][0, :, :].cpu().detach().numpy()
      for k in range(len(output[i]['masks'])):
        output_im2 = output[i]['masks'][k][0, :, :].cpu().detach().numpy()
        output_im2[output_im2>0.5] = 1
        output_im2[output_im2<0.5] = 0
        output_im = output_im+output_im2

      output_im[output_im>0.5] = 1
      output_im[output_im<0.5] = 0
      ax = figure.add_subplot(2,2, i+3)
      ax.imshow(output_im, cmap=cmap)

view_mask(targets, output, n=2)


In [ ]:
def IoU(y_real, y_pred):
  # Intersection over Union loss function
  intersection = y_real*y_pred
  #not_real = 1 - y_real
  #union = y_real + (not_real*y_pred)
  union = (y_real+y_pred)-(y_real*y_pred)
  return np.sum(intersection)/np.sum(union)

def dice_coef(y_real, y_pred, smooth=1):
  intersection = y_real*y_pred
  union = (y_real+y_pred)-(y_real*y_pred)
  return np.mean((2*intersection+smooth)/(union+smooth))

def confusion_matrix(y_true, y_pred):
  y_true= y_true.flatten()
  y_pred = y_pred.flatten()*2
  cm = y_true+y_pred
  cm = np.bincount(cm, minlength=4)
  tn, fp, fn, tp = cm
  return tp, fp, tn, fn

def get_f1_score(y_true, y_pred):
    """Return f1 score covering edge cases"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred)
    f1_score = (2 * tp) / ((2 * tp) + fp + fn)

    return f1_score     

In [ ]:
# Get IoU score for whole test set
IoU_scores_list = []
dice_coef_scores_list = []
f1_scores_list = []
skipped = 0
for images,targets in tqdm(data_loader_test):
  images = list(image.to(device) for image in images)
  targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

  model=model.double()
  model.eval()
  output = model(images)
  # print(targets)
  target_im = targets[0]['masks'][0].cpu().detach().numpy()
  for k in range(len(targets[0]['masks'])):
    target_im2 = targets[0]['masks'][k].cpu().detach().numpy()
    target_im2[target_im2>0.5] = 1
    target_im2[target_im2<0.5] = 0
    target_im = target_im+target_im2

  target_im[target_im>0.5] = 1
  target_im[target_im<0.5] = 0
  target_im = target_im.astype('int64')
  
  # Plot output (predicted) masks
  output_im = output[0]['masks'][0][0, :, :].cpu().detach().numpy()
  for k in range(len(output[0]['masks'])):
    output_im2 = output[0]['masks'][k][0, :, :].cpu().detach().numpy()
    output_im2[output_im2>0.5] = 1
    output_im2[output_im2<0.5] = 0
    output_im = output_im+output_im2

  output_im[output_im>0.5] = 1
  output_im[output_im<0.5] = 0
  output_im = output_im.astype('int64')

  if target_im.shape != output_im.shape:
    skipped+=1
    continue
  
  dice_coef_score = dice_coef(y_real=target_im, y_pred=output_im)
  dice_coef_scores_list.append(dice_coef_score)
  IoU_score = IoU(y_real=target_im, y_pred=output_im) 
  IoU_scores_list.append(IoU_score)
  f1_score = get_f1_score(target_im, output_im)
  f1_scores_list.append(f1_score)

print('mean IoU score for test set:', np.mean(IoU_scores_list))
print('mean Dice Coefficient score for test set:', np.mean(dice_coef_scores_list))
print('mean f1 score for test set:', np.mean(f1_scores_list))


In [ ]:
print('mean IoU score for test set:', np.mean(IoU_scores_list))

In [ ]:
for i in range(2):
  # plot target (true) masks
  target_im = targets[i]['masks'][0].cpu().detach().numpy()
  for k in range(len(targets[i]['masks'])):
    target_im2 = targets[i]['masks'][k].cpu().detach().numpy()
    target_im2[target_im2>0.5] = 1
    target_im2[target_im2<0.5] = 0
    target_im = target_im+target_im2

  target_im[target_im>0.5] = 1
  target_im[target_im<0.5] = 0
  # Plot output (predicted) masks
  output_im = output[i]['masks'][0][0, :, :].cpu().detach().numpy()
  for k in range(len(output[i]['masks'])):
    output_im2 = output[i]['masks'][k][0, :, :].cpu().detach().numpy()
    output_im2[output_im2>0.5] = 1
    output_im2[output_im2<0.5] = 0
    output_im = output_im+output_im2

  output_im[output_im>0.5] = 1
  output_im[output_im<0.5] = 0

IoU(y_real=target_im, y_pred=output_im) 


In [ ]:
len(output[0]['masks'])
im = output[0]['masks'][0][0, :, :].cpu().detach().numpy()
# im2 = outputs[0]['masks'][1][0, :, :].cpu().detach().numpy()
it = 0
for i in range(len(output[0]['masks'])):
  im2 = output[0]['masks'][i][0, :, :].cpu().detach().numpy()
  im2[im2>0.5] = 1
  im2[im2<0.5] = 0
  im = im+im2
  it+=1
print(it)
# im_new = np.concatenate((im, im2)) 
im[im>0.5] = 1
im[im<0.5] = 0
plt.imshow(im, cmap='Greys')
# outputs[0]['masks']

In [ ]:
# im = outputs[0]['masks'][20][0, :, :].cpu().detach().numpy()
# im[im>0.5] = 1
# im[im<0.5] = 0
# plt.imshow(im, cmap='Greys')

In [ ]:
# # outputs = model(images)
# outputs = [{k: v.to(device) for k, v in t.items()} for t in output]
# pred_score = list(outputs[0]['scores'].detach().numpy())
# pred_t = [pred_score.index(x) for x in pred_score if x>=threshold][-1]
# masks = (outputs[0]['masks']>0.5).squeeze().numpy()
# masks = masks[:pred_t+1]